<a href="https://colab.research.google.com/github/Spartan-119/A-B-Testing-Approach-for-Comparing-Performance-of-ML-Models/blob/main/a_b_testing_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing all the necessary packages
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 --upgrade
!pip install transformers --upgrade
!pip install tqdm
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00


In [2]:
# importing all the necessary libraries
import json
import os
import timeit
import collections
import time
from pprint import pprint
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, squad_convert_examples_to_features
from transformers.data.processors.squad import SquadV2Processor,SquadResult
from transformers.data.metrics.squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
    squad_evaluate,
)

In [3]:
DO_LOWER_CASE = True
NBEST_SIZE = 20
DOC_STRIDE = 128
MAX_SEQ_LENGTH = 384
MAX_QUERY_LENGTH = 64
MAX_ANSWER_LENGTH = 30
DATA_DIR = 'data/squad'
PREDICT_FILE = 'dev-v2.0.json'

BERT_MODEL_TYPE = 'bert'
BERT_MODEL_HF_PATH = 'twmkn9/bert-base-uncased-squad2'
BERT_OUTPUT_DIR = 'models/bert/twmkn9_bert-case-uncased-squad2'

DISTILBERT_MODEL_TYPE = 'distilbert'
DISTILBERT_MODEL_HF_PATH = 'twmkn9/distilbert-base-uncased-squad2'
DISTILBERT_OUTPUT_DIR = 'models/distilbert/twmkn9_distilbert-base-uncased-squad2'

# Downloading and Exploring the dataset

In [4]:
# downloading the dataset
!wget -P data/squad/ https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-09-11 21:27:48--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘data/squad/dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.05s   

2023-09-11 21:27:49 (85.7 MB/s) - ‘data/squad/dev-v2.0.json’ saved [4370528/4370528]



#### <i>Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

#### SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering.</i>


## Loading the DEV set using Hugging Face's data processors

I am going to make use of [Processors](https://huggingface.co/transformers/main_classes/processors.html) to facilitate basic processing tasks with some canonical NLP datasets. The processors can be used for loading datasets and converting their examples to features for direct use in the model. More specifically, we will be using the [SQuAD processors](https://huggingface.co/transformers/main_classes/processors.html#squad)

In [5]:
def to_list(tensor):
  return tensor.detach().cpu().tolist()

In [6]:
def load_and_cache_examples(model_name_or_path,
                            data_dir= DATA_DIR,
                            predict_file=PREDICT_FILE,
                            max_seq_length=MAX_SEQ_LENGTH,
                            doc_stride=DOC_STRIDE,
                            max_query_length=MAX_QUERY_LENGTH,
                            overwrite_cache=True):

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
    # Load data features from cache or dataset file
    input_dir = data_dir if data_dir else "."
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev",
            list(filter(None, model_name_or_path.split("/"))).pop(),
            str(max_seq_length),
        ),
    )

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file) and not overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:

        processor = SquadV2Processor()

        examples = processor.get_dev_examples(data_dir, filename=predict_file)

        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset="pt",
            threads=1,
        )


    return dataset, examples, features

In [7]:
dataset, examples, features = load_and_cache_examples(BERT_MODEL_HF_PATH)

add example index and unique id: 100%|██████████| 11873/11873 [00:00<00:00, 832188.16it/s]


In [8]:
print(f'There are {len(examples)} examples in the dev dataset.')

There are 11873 examples in the dev dataset.


This list of examples contains objects of type transformers.data.processors.squad.SquadExample.
We use the functionbelow to extract the information we want from such objects.
More specifically: 'qid', 'question_text', 'context_text' and 'answer'.

I will first create some extra variables to help on manipulation of data.

In [9]:
# generating some maps to help identify examples of interest.
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]

And also, the function below to help on extracting information given a `qid` (question unique identifier)

In [10]:
def display_example(qid: str) -> None:
  idx = qid_to_example_index[qid]
  q = examples[idx].question_text
  c = examples[idx].context_text
  a = [answer['text'] for answer in examples[idx].answers]

  print(f'Examples {idx} of {len(examples)}\n------------------')
  print(f'Q: {q}\n')
  print('Context:')
  pprint(c)
  print(f'\nTrue Answers:\n{a}')

## Positive Example

50% of the examples in the test set are questionst hat have answers contained within their corresponding passage. In these cases, up to 5 possible correct answers are provided. Such answers must come directly from the passage, we will see later, however, that there are several ways to arrive at a "correct" answer.

In [11]:
display_example(answer_qids[2456])

Examples 4959 of 11873
------------------
Q: It is now possible to convert old relative ages into what type of ages using isotopic dating?

Context:
('At the beginning of the 20th century, important advancement in geological '
 'science was facilitated by the ability to obtain accurate absolute dates to '
 'geologic events using radioactive isotopes and other methods. This changed '
 'the understanding of geologic time. Previously, geologists could only use '
 'fossils and stratigraphic correlation to date sections of rock relative to '
 'one another. With isotopic dates it became possible to assign absolute ages '
 'to rock units, and these absolute dates could be applied to fossil sequences '
 'in which there was datable material, converting the old relative ages into '
 'new absolute ages.')

True Answers:
['absolute ages', 'rock units', 'new absolute']


## Negative Example

The remaining 50% of the questions in the test set do not have an answer. This is important as in a real life Q&A system, the model needs to learn when **NOT TO ANSWER.**

In [12]:
display_example(no_answer_qids[1235])

Examples 2520 of 11873
------------------
Q: What is difficult with a satellite-to-noise ratio?

Context:
('Oxygen presents two spectrophotometric absorption bands peaking at the '
 'wavelengths 687 and 760 nm. Some remote sensing scientists have proposed '
 'using the measurement of the radiance coming from vegetation canopies in '
 'those bands to characterize plant health status from a satellite platform. '
 'This approach exploits the fact that in those bands it is possible to '
 "discriminate the vegetation's reflectance from its fluorescence, which is "
 'much weaker. The measurement is technically difficult owing to the low '
 'signal-to-noise ratio and the physical structure of vegetation; but it has '
 'been proposed as a possible method of monitoring the carbon cycle from '
 'satellites on a global scale.')

True Answers:
[]


## Metrics for Q&A Systems

When measuring the performance of a machine learning system, we need to think about both **model** and **customer metrics.**

Q&A systems are usually measured by two dominant metrics: **F1** and **Exact Match (EM)**. They are computed on individual question and answer **pairs**. When multiple correct answers are available for a given question the maximum score over all possible correct answers is computed. Overall EM and F1 scores are computed for a model by averaging over the individual example scores.

### Exact Match:

For each Q&A pair, if the characters of the model's prediction are an exact matvh of the characters of any of the True Answer(s), **EM = 1**, else, **EM = 0**. This is a strict all-or-nothing metric, which may have little value for final customers of a Q&A system. It may be beneficial only when assessing against a negative example; if the model predicts any text at all, it automatically receives a 0 for that example.

### F1 Score:

Almost all classification problems rely on F1 score to measure the model performance. It is mostly appropriate when we care equally about precision and recall. On an QnA system, however, it is computed over the individual words in the prediction against those in the True answer. The number of shared words between the prediction and the trust is the basis of F1 score. While **precision** is the ratio between the number of shared words to the total number of words in the prediction; **Recall** is the ratio of the number of shared words to the toal number of words in the ground truth.

### Latency

Latency is an important metric for ML systems. in the QnA example, it is of the utmost importance when the system is used in a conversational application. <br> For Example, Alexa and Google home are devices that have very strict latency constraints as the users expect an answer within a few seconds or macroseconds after the question is asked. When updating models we should take this dimension according to the application of the system.

### Answer Rate

In the QnA systems, models that attempt to answer every question are often perceived inaccurate. The system should only provide an output when it is confident enough to do so. In other words, when the probabiliries of predictions are above a certain threshold. <br> In some applications, a model should be able to say *I don't know* or *The context has not enough information to answer the question.*

# Q&A Models

Question and Answering make use of Large Language Models (LLMs) as any other classification problem in NLP. The main difference relies on how the input and the output is provided to the model. Generally speaking models are trained to match the true answer to the question as they are provided together as an input to the model.

# Bert

BERT is a neural approach to pre-train language representations which obtains near SOTA results on a wide array of NLP tasks, including SQuAD Question Answering dataset <br><br>

Developed in 2019, BERT achieves 80.422% in the EM score and 83.118% in the F1 score.<br><br>

BERT-base has 110 million parameters and BERT-large has 340 million parameters.

## Model Parameters comparison

As LLMs were developed, the amount of parameters in these models have grown exponentially. Although this improves model performance, it comes at a cost: Latency. As it will be discussed, for use cases where inference is done on batches that may have less impact, however, on real time systems such as voice assistants or web search, latency plays a major role on deciding whether one model is better than the other.

# BERT Input

[CLS] context [SEP] question [SEP] [PAD] [PAD] [PAD]

**context** = "The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations."

**question** = "What organization is the IPCC a part of?"

**after being merged by the tokenizer**:
```
"[CLS] The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations. [SEP] What organization is the IPCC a part of? [SEP] [PAD] [PAD] [PAD]"
```


**token-id format**:

[101, 1109, 11300, 2758, 24472, 15595, 20339, 1113, 13540, 9091, 113, 14274, 12096, 114, 1110, 170, 3812,
 9455, 2758, 24472, 15595, 1404, 1223, 1103, 22105, 1104, 1103, 1244, 3854, 119, 102, 1327, 2369, 1110, 1103,
 14274, 12096, 170, 1226, 1104, 136, 102, 0, 0, 0]

# Loading Pre-Trained BERT from Hugging Face's repository.

In [13]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_HF_PATH, use_fast = False)
model = AutoModelForQuestionAnswering.from_pretrained(BERT_MODEL_HF_PATH)

Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Utility Functions

Given a question_id, model and tokenizer, we get an answer text. In here we get the maximum probability of beginning and end for the answer in the softmax output.

In [14]:
def get_prediction(qid: str, model: AutoModelForQuestionAnswering, tokenizer: AutoTokenizer):
  # given a question_id (qas_id or qid), load the example, get the model outputs and generate an answer
  question = examples[qid_to_example_index[qid]].question_text
  context = examples[qid_to_example_index[qid]].context_text

  inputs = tokenizer.encode_plus(question, context, return_tensors = 'pt')

  outputs = model(**inputs)

  answer_start = torch.argmax(outputs[0])       # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start: answer_end]))

  return answer

I am creatinga simple function that given an example list, it extracts the gold answers.

In [15]:
def get_gold_answers(example):
  """
  helper function that retrieves all possible true answers from a squad2.0 example.
  """

  gold_answers = [answer['text'] for answer in example.answers if answer['text']]

  # if gold_answers doesn't exist, it's because this is a negative example.
  # the only correct answer is an empty string then in this case.
  if not gold_answers:
    gold_answers = [""]

  return gold_answers

For metrics like EM, we need to make sure that texts are normalized so we can compare on a character level.

In [16]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s: str):
  """
  removing articles and punctuation, and standardizing whitespace are all typical text processing steps.
  """
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

# Metrics Calculation

## Exact Match (EM)

In [17]:
def compute_exact_match(prediction, truth):
  return int(normalize_text(prediction) == normalize_text(truth))

## F1 Score

In [18]:
def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, else 0
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) ==  0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)

Computing EM and F1 for an example with a gold answer

In [19]:
prediction = get_prediction(answer_qids[1303], model, tokenizer, )
example = examples[qid_to_example_index[answer_qids[1303]]]

gold_answers = get_gold_answers(example)

em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

print(f"Question: {example.question_text}")
print(f"Prediction: {prediction}")
print(f"True Answers: {gold_answers}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: What measurement do scientists used to determine the quality of water?
Prediction: biochemical oxygen demand
True Answers: ['biochemical oxygen demand', 'biochemical oxygen demand', "measuring the water's biochemical oxygen demand", 'biochemical oxygen demand', "measuring the water's biochemical oxygen demand"]
EM: 1 	 F1: 1.0


Now let's try and compute an example without answer

In [20]:
prediction = get_prediction(no_answer_qids[1254], model, tokenizer)
example = examples[qid_to_example_index[no_answer_qids[1254]]]

gold_answers = get_gold_answers(example)

em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

print(f"Question: {example.question_text}")
print(f"Prediction: {prediction}")
print(f"True Answers: {gold_answers}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: What happened 3.7-2 billion years ago?
Prediction: [CLS]
True Answers: ['']
EM: 0 	 F1: 0


Both metrics are zero, this model does not correctly assess that this question is unanswerable. It predicts the [CLS] token (it means it considers the entire context as an answer to the question)

## Putting it all together

In [21]:
def get_answers_metrics(
    model: AutoModelForQuestionAnswering,
    tokenizer: AutoTokenizer,
    answer_qids = answer_qids,
    examples = examples):

  answers_arr = []
  start_time = time.time()
  errors = []

  for qid in tqdm(answer_qids):
    try:
      prediction = get_prediction(qid, model, tokenizer)
      example = examples[qid_to_example_index[qid]]

      gold_answers = get_gold_answers(example)

      em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
      f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

      result_dict = {}
      result_dict["qid"] = qid
      result_dict["question"] = example.question_text
      result_dict["prediction"] = prediction
      result_dict["true_answers"] = ';'.join(gold_answers)
      result_dict["f1"] = f1_score
      result_dict["em"] = em_score
      answers_arr.append(result_dict)
    except:
      errors.append(qid)

  end_time = time.time()

  return pd.DataFrame(answers_arr), end_time - start_time, errors

In [22]:
metrics_df, total_time, errors = get_answers_metrics(model, tokenizer, answer_qids[: 100])

100%|██████████| 100/100 [00:36<00:00,  2.71it/s]


In [23]:
metrics_df.head()

,qid,question,prediction,true_answers,f1,em
0,56ddde6b9a695914005b9628,In what country is Normandy located?,france,France;France;France;France,1.0,1
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,10th and 11th centuries,10th and 11th centuries;in the 10th and 11th c...,1.0,1
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,"denmark , iceland and norway","Denmark, Iceland and Norway;Denmark, Iceland a...",1.0,1
3,56ddde6b9a695914005b962b,Who was the Norse leader?,rollo,Rollo;Rollo;Rollo;Rollo,1.0,1
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,10th,10th century;the first half of the 10th centur...,1.0,1


In [24]:
metrics_df['f1'].mean()

0.7866849965752404

In [25]:
metrics_df['em'].mean()

0.72

## Improving measurement functions through model thresholding

When we tokenize a question and context, and we pass it to the model, the output consists of two probabilities (logits). One is the start of the answer span, the other for the end of the answer span. <br><br>

Every tokent that is passed to the model is assigned a logit, and tokens corresponding to the question itself. <br><br>

Let's have a look at what this means, using a previous question ("What happened 3.7 - 2 billion years ago?"):

In [26]:
inputs = tokenizer.encode_plus(example.question_text, example.context_text, return_tensors = 'pt')
output = model(**inputs)

Looking below, we can observe how large is the first position of the array, this is the [CLS] token position. This has a strong probability that this question has no answer, but we answered it anyway.

In [27]:
start_logits = output.start_logits
end_logits = output.end_logits

In [28]:
start_logits

tensor([[  5.1171,  -8.3404,  -9.2660,  -8.0987,  -9.1736,  -9.5905,  -9.6239,
          -9.4974,  -9.7725,  -9.9778, -10.1417,  -9.6171,  -8.7191,  -3.9945,
          -5.9036,  -6.7555,  -8.8160,  -7.9155,  -8.7225,  -9.7324,  -9.5704,
         -10.2445,  -9.0550,  -7.5630,  -9.8712, -10.0757,  -7.6297,  -7.4481,
          -6.5011,  -9.7018, -10.0513,  -9.4285,  -8.6840, -10.2489,  -9.9774,
          -8.0076,  -7.7707,  -8.0321,  -6.8884,  -6.6425,  -6.5631,  -9.4518,
          -8.6434,  -9.5573,  -9.8626,  -9.7323,  -7.0476,  -4.5778,  -6.4825,
          -6.7371,  -6.9433,  -9.1568,  -6.9630,  -8.9800,  -6.9723,  -7.3322,
          -5.2532,  -9.6134,  -9.4807, -10.0298,  -9.8842,  -8.8732,  -7.9342,
          -8.2085,  -8.0398,  -7.8698,  -7.2027,  -9.6577,  -9.1055,  -9.7395,
          -7.7438,  -9.5543,  -9.0663,  -9.2180,  -9.8046,  -9.6983,  -8.9082,
          -6.9894,  -6.7306,  -7.6100,  -6.7132,  -8.6220,  -9.4562,  -8.3209,
          -5.3854,  -6.1157,  -6.8288,  -8.7172,  -9

Our model gets predictions by selecting the start and end tokens with the largest logits. It would be more sensible to choose any sensible start + end combinations as possible to answer the question.

These combinations can be scored independently and the one with the highest score would be considered the best answer.

A possible (candidate) answer is scored as the sum of its start and end logits

## Calculating possible combinations

We start by taking the n largest start and end logits. Any sensible combination can be considered an answer, however, some consistency checks must first be performed.

For instance:
    
    - End token must fall after the start token
    - Candidate answers wherein the start or end tokens are associated with question tokens

[CLS] is not removed from the answers as it can indicate null answer

In [29]:
# convert our start and end logit tensors to lists
start_logits = to_list(start_logits)[0]
end_logits = to_list(end_logits)[0]

In [30]:
# sort our start and end logits from the largest to the smallest, keeping track of the index
start_idx_and_logit = sorted(enumerate(start_logits), key = lambda x: x[1], reverse = True)
end_idx_and_logit = sorted(enumerate(end_logits), key = lambda x: x[1], reverse = True)

In [31]:
# select the top n (in this case, 5)
print(start_idx_and_logit[: 5])
print(end_idx_and_logit[: 5])

[(0, 5.117069244384766), (111, 1.3977384567260742), (104, 0.6027610301971436), (106, -1.1286728382110596), (113, -1.7321617603302002)]
[(0, 6.168288230895996), (119, 3.2872838973999023), (109, 0.9794851541519165), (135, 0.30854111909866333), (116, -0.20684576034545898)]


The null answer token (index 0) is in the top five of both the start and end logit lists.

In order to eventually predict a text answer (or an empty string), we need to keep track of the indexes which will be use to pull the corresponding token ids later on. We'll also need to identify which indexes correspond to the question tokens, so we can ensure we don't alow a non-sensical prediction.

In [32]:
start_indexes = [idx for idx, logit in start_idx_and_logit[: 5]]
end_indexes = [idx for idx, logit in end_idx_and_logit[: 5]]

In [33]:
# convert the token ids from a tensor to a list
tokens = to_list(inputs['input_ids'])[0]

In [34]:
# question tokens are defined as those between the CLS token (101, at position 0) and first SEP (102) token
question_indexes = [i + 1 for i, token in enumerate(tokens[1: tokens.index(102)])]
question_indexes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [35]:
# keep track of all preliminary predictions
PrelimPrediction = collections.namedtuple(
    "PrelimPrediction", ["start_index", "end_index", "start_logit", "end_logit"]
)

We will generate a list of candidate predictions by looping through all combinations of the start and end token indexes, excluding non-sensical combinations.

In [36]:
prelim_preds = []

for start_index in start_indexes:
  for end_index in end_indexes:
    # throw out invalid predictions
    if start_index in question_indexes:
      continue
    if end_index in question_indexes:
      continue
    if end_index < start_index:
      continue

    prelim_preds.append(
        PrelimPrediction(
            start_index = start_index,
            end_index = end_index,
            start_logit = start_logits[start_index],
            end_logit = end_logits[end_index]
        )
    )

With a list of sensible candidate predictions, it's time to score them now.

For a candidate answer, score = start_logit + end_logit. Below, we sort our candidate predictions by their score.

In [37]:
# sort preliminary predictions by their score
prelim_preds = sorted(prelim_preds, key = lambda x: (x.start_logit + x.end_logit), reverse = True)
print(prelim_preds[: 5])

[PrelimPrediction(start_index=0, end_index=0, start_logit=5.117069244384766, end_logit=6.168288230895996), PrelimPrediction(start_index=0, end_index=119, start_logit=5.117069244384766, end_logit=3.2872838973999023), PrelimPrediction(start_index=0, end_index=109, start_logit=5.117069244384766, end_logit=0.9794851541519165), PrelimPrediction(start_index=0, end_index=135, start_logit=5.117069244384766, end_logit=0.30854111909866333), PrelimPrediction(start_index=0, end_index=116, start_logit=5.117069244384766, end_logit=-0.20684576034545898)]


We need to convert our preliminary predictions into actual text (or the empty string, if null). We will keep track of text predictions we've seen, because different token combinations can result in the same text prediction and we only want to keep the one with the highest score (we're looping in descending score order). Finally, we'll trim this list down to the best 5 predictions.

In [38]:
# keep track of all the best predictions
BestPrediction = collections.namedtuple( # pylint: disable = invalid-name
                                       "BestPrediction", ["text", "start_logit", "end_logit"]
                                        )

In [39]:
nbest = []
seen_predictions = []

for pred in prelim_preds:
  # for now we only care about the top 5 best predictions
  if len(nbest) >= 5:
    break

  # loop through the predictions according to their start index
  if pred.start_index > 0: # non-null answers have start_index > 0
    text = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(
            tokens[pred.start_index: pred.end_index + 1]
        )
    )

    # clean the whitespace
    text = text.strip()
    text = " ".join(text.split())

    if text in seen_predictions:
      continue

    # flag this text as being seen - if we see it again, don't add it to the nbest list
    seen_predictions.append(text)

    # add this text prediction to a pruned list of the top 5 best predictions
    nbest.append(BestPrediction(text = text, start_logit = pred.start_logit, end_logit = pred.end_logit))

In [40]:
nbest

[BestPrediction(text='free oxygen began to outgas from the oceans', start_logit=1.3977384567260742, end_logit=3.2872838973999023),
 BestPrediction(text='when such oxygen sinks became saturated , free oxygen began to outgas from the oceans', start_logit=0.6027610301971436, end_logit=3.2872838973999023),
 BestPrediction(text='oxygen sinks became saturated , free oxygen began to outgas from the oceans', start_logit=-1.1286728382110596, end_logit=3.2872838973999023),
 BestPrediction(text='free oxygen began to outgas from the oceans 3 – 2 . 7 billion years ago , reaching 10 % of its present level', start_logit=1.3977384567260742, end_logit=0.30854111909866333),
 BestPrediction(text='when such oxygen sinks became saturated', start_logit=0.6027610301971436, end_logit=0.9794851541519165)]

At this point, we have a neat list of the top 5 best predictions for this question, let's now also add the null answer.

In [41]:
# and don't forget -- include the null answer!
nbest.append(BestPrediction(text = "", start_logit = start_logits[0], end_logit = end_logits[0]))
nbest

[BestPrediction(text='free oxygen began to outgas from the oceans', start_logit=1.3977384567260742, end_logit=3.2872838973999023),
 BestPrediction(text='when such oxygen sinks became saturated , free oxygen began to outgas from the oceans', start_logit=0.6027610301971436, end_logit=3.2872838973999023),
 BestPrediction(text='oxygen sinks became saturated , free oxygen began to outgas from the oceans', start_logit=-1.1286728382110596, end_logit=3.2872838973999023),
 BestPrediction(text='free oxygen began to outgas from the oceans 3 – 2 . 7 billion years ago , reaching 10 % of its present level', start_logit=1.3977384567260742, end_logit=0.30854111909866333),
 BestPrediction(text='when such oxygen sinks became saturated', start_logit=0.6027610301971436, end_logit=0.9794851541519165),
 BestPrediction(text='', start_logit=5.117069244384766, end_logit=6.168288230895996)]

The null answer is scored as the sum of the start_logit and end_logit associated with the [CLS] token.

The last step is to compute the null score - more specifically, the difference between the null score and the best non-null score as shown below.

In [42]:
# compute the null score as the sum of the [CLS] token logits
score_null = start_logits[0] + end_logits[0]
score_null

11.285357475280762

In [43]:
nbest[0].start_logit + nbest[0].end_logit

4.685022354125977

In [44]:
# compute the difference between the null score and tehbest non-null score
score_diff = score_null - nbest[0].start_logit - nbest[0].end_logit
score_diff

6.600335121154785

# SQuAD Evaluation

In [45]:
def evaluate(model_name_or_path,
             dataset,
             output_dir,
             per_gpu_eval_batch_size = 12,
             n_gpu = 1,
             model_type = BERT_MODEL_TYPE,
             do_lower_case = DO_LOWER_CASE,
             nbest_size = NBEST_SIZE,
             max_answer_length = MAX_ANSWER_LENGTH,
             null_score_diff_threshold = 0.0):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast = False)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

    model.to(device)

    eval_batch_size = per_gpu_eval_batch_size * max(1, n_gpu)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler = eval_sampler, batch_size = eval_batch_size)

    # multi-gpu evaluate
    if n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
        model = torch.nn.DataParallel(model)

    all_results = []
    start_time = timeit.default_timer()

    for batch in tqdm(eval_dataloader, desc = "Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            if model_type in ["xlm", "roberta", "distilbert", "camembert", "bart", "longformer"]:
                del inputs["token_type_ids"]

            feature_indices = batch[3]

            outputs = model(**inputs)

        for i, feature_index in enumerate(feature_indices):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs.to_tuple()]

            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    print(f"Evaluation done in total {evalTime} seconds ({evalTime / len(dataset)} seconds per example)")

    # compute predictions
    os.makedirs(output_dir, exist_ok = True)

    output_prediction_file = os.path.join(output_dir, "predictions.json")
    output_nbest_file = os.path.join(output_dir, "nbest_predictions.json")

    output_null_log_odds_file = os.path.join(output_dir, "null_odds.json")

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        nbest_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,
        True,
        null_score_diff_threshold,
        tokenizer,
    )

    # compute the F1 and EM scores
    results = squad_evaluate(examples, predictions)

    results.update({"eval_time": evalTime, "prediction_time": evalTime / len(dataset)})

    return results

In [46]:
result = evaluate(BERT_MODEL_HF_PATH, dataset, BERT_OUTPUT_DIR)
result

Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Evaluating: 100%|██████████| 1020/1020 [04:58<00:00,  3.42it/s]


Evaluation done in total 298.374268584 seconds (0.02439292581621975 seconds per example)


OrderedDict([('exact', 72.3658721468879),
             ('f1', 75.83107045305708),
             ('total', 11873),
             ('HasAns_exact', 72.80701754385964),
             ('HasAns_f1', 79.74735146578041),
             ('HasAns_total', 5928),
             ('NoAns_exact', 71.9259882253995),
             ('NoAns_f1', 71.9259882253995),
             ('NoAns_total', 5945),
             ('best_exact', 72.3658721468879),
             ('best_exact_thresh', 0.0),
             ('best_f1', 75.8310704530571),
             ('best_f1_thresh', 0.0),
             ('eval_time', 298.374268584),
             ('prediction_time', 0.02439292581621975)])

The first three blocks of the results output are pretty straightforward. EM and F1 scores are reported over


*   the full dev set,
*   the set of positive examples,
*   the set of negative examples.

This can provide some insight into whether a model is performing adequately on both answer and no-answer questions.



As per below results, we see that two sections are zero. They depend on setting a threshold for the model so it knows when to prefer a null answer over an actual answer.

In other words, we should predict a null answer for a given example if that example's score difference is above a certain threshold. What should be that threshold? How should we compute it? They give us a recipe: select the threshold that maximizes F1.

We will leverage the files created by the evaluate function above to retrieve the optimal threshold.

In [47]:
# load the predictions we generated earlier
filename = BERT_OUTPUT_DIR + '/predictions.json'
preds = json.load(open(filename, 'rb'))

# load the null score differences we generated earlier
filename = BERT_OUTPUT_DIR + '/null_odds.json'
null_odds = json.load(open(filename, 'rb'))

In [48]:
# the default threshold is set to 1.0 -- we'll leave it there for now
results_default_thresh = squad_evaluate(examples,
                                        preds,
                                        no_answer_probs=null_odds,
                                        no_answer_probability_threshold=1.0)

pprint(results_default_thresh)

OrderedDict([('exact', 72.3658721468879),
             ('f1', 75.83107045305708),
             ('total', 11873),
             ('HasAns_exact', 72.80701754385964),
             ('HasAns_f1', 79.74735146578041),
             ('HasAns_total', 5928),
             ('NoAns_exact', 71.9259882253995),
             ('NoAns_f1', 71.9259882253995),
             ('NoAns_total', 5945),
             ('best_exact', 73.66293270445549),
             ('best_exact_thresh', -3.76619291305542),
             ('best_f1', 76.85604013447075),
             ('best_f1_thresh', -3.4150748252868652)])


The first three blocks have identical values as in our initial evaluation because they are based on the default threshold (which is currently 1.0). However, the values in the fourth block have been updated by taking into account the null_odds information. When a given example's score_diff is greater than the threshold, the prediction is flipped to a null answer which affects the overall EM and F1 scores.

In [49]:
best_f1_thresh = -3.415079116821289
results_f1_thresh = squad_evaluate(examples,
                                   preds,
                                   no_answer_probs=null_odds,
                                   no_answer_probability_threshold=best_f1_thresh)

pprint(results_f1_thresh)

OrderedDict([('exact', 73.6208203486903),
             ('f1', 76.84761766331809),
             ('total', 11873),
             ('HasAns_exact', 69.31511470985156),
             ('HasAns_f1', 75.77796297513085),
             ('HasAns_total', 5928),
             ('NoAns_exact', 77.91421362489487),
             ('NoAns_f1', 77.91421362489487),
             ('NoAns_total', 5945),
             ('best_exact', 73.66293270445549),
             ('best_exact_thresh', -3.76619291305542),
             ('best_f1', 76.85604013447075),
             ('best_f1_thresh', -3.4150748252868652)])


We can see that metrics for NoAns have increased significantly. The downside is that we lose some ground in how well our model correctly predicts HasAns examples. Overall, however, we see a net increase of a couple points in both EM and F1 scores. This demonstrates that computing null scores and properly using a null threshold significantly increases QA performance on the SQuAD2.0 dev set with almost no additional work.

# Putting it all together

In [50]:
def get_qa_inputs(example, tokenizer):
    # load the example, convert to inputs, get model outputs
    question = example.question_text
    context = example.context_text
    return tokenizer.encode_plus(question, context, return_tensors='pt')

def get_clean_text(tokens, tokenizer):
    text = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(tokens)
        )
    # Clean whitespace
    text = text.strip()
    text = " ".join(text.split())
    return text

def prediction_probabilities(predictions):

    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    all_scores = [pred.start_logit+pred.end_logit for pred in predictions]
    return softmax(np.array(all_scores))

In [51]:
def preliminary_predictions(start_logits, end_logits, input_ids, nbest):
    # convert tensors to lists
    start_logits = to_list(start_logits)[0]
    end_logits = to_list(end_logits)[0]
    tokens = to_list(input_ids)[0]

    # sort our start and end logits from largest to smallest, keeping track of the index
    start_idx_and_logit = sorted(enumerate(start_logits), key=lambda x: x[1], reverse=True)
    end_idx_and_logit = sorted(enumerate(end_logits), key=lambda x: x[1], reverse=True)

    start_indexes = [idx for idx, logit in start_idx_and_logit[:nbest]]
    end_indexes = [idx for idx, logit in end_idx_and_logit[:nbest]]

    # question tokens are between the CLS token (101, at position 0) and first SEP (102) token
    question_indexes = [i+1 for i, token in enumerate(tokens[1:tokens.index(102)])]

    # keep track of all preliminary predictions
    PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "PrelimPrediction", ["start_index", "end_index", "start_logit", "end_logit"]
    )
    prelim_preds = []
    for start_index in start_indexes:
        for end_index in end_indexes:
            # throw out invalid predictions
            if start_index in question_indexes:
                continue
            if end_index in question_indexes:
                continue
            if end_index < start_index:
                continue
            prelim_preds.append(
                PrelimPrediction(
                    start_index = start_index,
                    end_index = end_index,
                    start_logit = start_logits[start_index],
                    end_logit = end_logits[end_index]
                )
            )
    # sort prelim_preds in descending score order
    prelim_preds = sorted(prelim_preds, key=lambda x: (x.start_logit + x.end_logit), reverse=True)
    return prelim_preds

In [52]:
def best_predictions(prelim_preds, nbest, tokenizer):
    # keep track of all best predictions

    # This will be the pool from which answer probabilities are computed
    BestPrediction = collections.namedtuple(
        "BestPrediction", ["text", "start_logit", "end_logit"]
    )
    nbest_predictions = []
    seen_predictions = []
    for pred in prelim_preds:
        if len(nbest_predictions) >= nbest:
            break
        if pred.start_index > 0: # non-null answers have start_index > 0

            toks = tokens[pred.start_index : pred.end_index+1]
            text = get_clean_text(toks, tokenizer)

            # if this text has been seen already - skip it
            if text in seen_predictions:
                continue

            # flag text as being seen
            seen_predictions.append(text)

            # add this text to a pruned list of the top nbest predictions
            nbest_predictions.append(
                BestPrediction(
                    text=text,
                    start_logit=pred.start_logit,
                    end_logit=pred.end_logit
                    )
                )

    # Add the null prediction
    nbest_predictions.append(
        BestPrediction(
            text="",
            start_logit=start_logits[0],
            end_logit=end_logits[0]
            )
        )
    return nbest_predictions

In [53]:
def compute_score_difference(predictions):
    """ Assumes that the null answer is always the last prediction """
    score_null = predictions[-1].start_logit + predictions[-1].end_logit
    score_non_null = predictions[0].start_logit + predictions[0].end_logit
    return score_null - score_non_null

In [54]:
def get_robust_prediction(example, tokenizer, nbest=10, null_threshold=1.0):

    inputs = get_qa_inputs(example, tokenizer)
    output = model(**inputs)

    # get sensible preliminary predictions, sorted by score
    prelim_preds = preliminary_predictions(output.start_logits,
                                           output.end_logits,
                                           inputs['input_ids'],
                                           nbest)

    # narrow that down to the top nbest predictions
    nbest_preds = best_predictions(prelim_preds, nbest, tokenizer)

    # compute the probability of each prediction - nice but not necessary
    probabilities = prediction_probabilities(nbest_preds)

    # compute score difference
    score_difference = compute_score_difference(nbest_preds)

    # if score difference > threshold, return the null answer
    if score_difference > null_threshold:
        return "", probabilities[-1]
    else:
        return nbest_preds[0].text, probabilities[0]

We are not able to detect 'No Answer'

In [55]:
print(example.question_text)
get_robust_prediction(example, tokenizer, nbest=10, null_threshold=best_f1_thresh)

What happened 3.7-2 billion years ago?


('', 0.9975693374241689)

# Using A/B Testing to make model deployment decisions

Now we have all the tools to make decisions on whether a model is better than the other.

## Home assistant device

We have been tasked to evaluate whether a new model recently trained is better than our current model in production

While talking to our business we are able to enumerate our priorities in order of importance:

1) Our users should receive their answer quickly, and the threshold for the isolated prediction of the model is 0.01 seconds 2) Its important to not surface answers when questions are either malformed or don't have enough context to be answered 3) Accuracy is important to the level where uses care about factuality without too much concern with answers having exact words as their ground truth

We are looking for the f1 score of "has answer" and "no answer" separately, so we can weight them according to the requirements

Also, we want to measure latency as: time to predict 10000 questions from our user

Given this scenario we write the following objective function:

In [56]:
def home_assistant_score(has_answer_f1, no_answer_f1, prediction_time):
    return ((0.2 * has_answer_f1 + 0.3 * no_answer_f1) - (0.5 * (10000 * prediction_time)))

We assign very high weight (0.5) to our prediction time, while dividing the remaining between our both F1 scores, giving slightly higher importance to "No Answer" scores

Lets evaluate now our BERT model given the above Objective function

In [57]:
result = evaluate(BERT_MODEL_HF_PATH, dataset, BERT_OUTPUT_DIR)
result

Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Evaluating: 100%|██████████| 1020/1020 [05:17<00:00,  3.21it/s]


Evaluation done in total 317.52474338 seconds (0.025958530361347286 seconds per example)


OrderedDict([('exact', 72.3658721468879),
             ('f1', 75.83107045305708),
             ('total', 11873),
             ('HasAns_exact', 72.80701754385964),
             ('HasAns_f1', 79.74735146578041),
             ('HasAns_total', 5928),
             ('NoAns_exact', 71.9259882253995),
             ('NoAns_f1', 71.9259882253995),
             ('NoAns_total', 5945),
             ('best_exact', 72.3658721468879),
             ('best_exact_thresh', 0.0),
             ('best_f1', 75.8310704530571),
             ('best_f1_thresh', 0.0),
             ('eval_time', 317.52474338),
             ('prediction_time', 0.025958530361347286)])

In [58]:
print(f"Model {BERT_MODEL_TYPE} score is {home_assistant_score(result['HasAns_f1'], result['NoAns_f1'], result['prediction_time'])}")

Model bert score is -92.26538504596049


Ok, now we have a number that quantifies our business requirements.

One of our data scientists, have found a 'faster' version of BERT, called 'distilled bert' that has 40% less parameters and is 60% faster while preserving 95% of BERT's performance

Lets now evaluate this model using our pipeline:

In [59]:
result_distilled = evaluate(model_name_or_path=DISTILBERT_MODEL_HF_PATH, dataset=dataset, output_dir=DISTILBERT_OUTPUT_DIR, model_type=DISTILBERT_MODEL_TYPE)
result_distilled

Evaluating: 100%|██████████| 1020/1020 [02:40<00:00,  6.36it/s]


Evaluation done in total 160.2621404219999 seconds (0.013101875443263564 seconds per example)


OrderedDict([('exact', 66.25958056093658),
             ('f1', 69.66994428499025),
             ('total', 11873),
             ('HasAns_exact', 68.91025641025641),
             ('HasAns_f1', 75.74076391627662),
             ('HasAns_total', 5928),
             ('NoAns_exact', 63.61648444070648),
             ('NoAns_f1', 63.61648444070648),
             ('NoAns_total', 5945),
             ('best_exact', 66.25958056093658),
             ('best_exact_thresh', 0.0),
             ('best_f1', 69.66994428499046),
             ('best_f1_thresh', 0.0),
             ('eval_time', 160.2621404219999),
             ('prediction_time', 0.013101875443263564)])

In [60]:
print(f"Model {DISTILBERT_MODEL_TYPE} score is {home_assistant_score(result_distilled['HasAns_f1'], result_distilled['NoAns_f1'], result_distilled['prediction_time'])}")

Model distilbert score is -31.276279100850545


The Distilled Version of BERT achieves exactly what it was described, it produces predictions in almost half of the speed of the traditional BERT with very minimal impacts to performance

The recommendation in the above case is to switch BERT model for the DistilledBERT as it improves business metrics by 3x

------------------------------------------------------------------------------------------end of notebook------------------------------------------------------------------------------------------